In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller

In [ ]:
tickers = ['TSLA', 'BND', 'SPY']
start_date = "2015-07-01"
end_date = "2025-07-31"

# Download data for all tickers
data = {ticker: yf.download(ticker, start=start_date, end=end_date) for ticker in tickers}

# Create DataFrame of Adjusted Close prices
adj_close = pd.DataFrame({ticker: df['Adj Close'] for ticker, df in data.items()})

In [ ]:
print("\nBasic Statistics:")
print(adj_close.describe())

# Missing values check
print("\nMissing Values Count:")
print(adj_close.isnull().sum())

# Fill missing values (Forward Fill)
adj_close = adj_close.fillna(method='ffill')

# Daily percentage returns
daily_returns = adj_close.pct_change().dropna()

In [ ]:
# Adjusted close prices over time
plt.figure(figsize=(12, 6))
for ticker in tickers:
    plt.plot(adj_close.index, adj_close[ticker], label=ticker)
plt.title("Adjusted Close Prices Over Time")
plt.xlabel("Date")
plt.ylabel("Price ($)")
plt.legend()
plt.grid(True)
plt.show()

# Daily returns
plt.figure(figsize=(12, 6))
for ticker in tickers:
    plt.plot(daily_returns.index, daily_returns[ticker], label=f"{ticker} Daily Return")
plt.title("Daily Percentage Change")
plt.xlabel("Date")
plt.ylabel("Daily Return")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
rolling_volatility = daily_returns.rolling(window=30).std()

plt.figure(figsize=(12, 6))
for ticker in tickers:
    plt.plot(rolling_volatility.index, rolling_volatility[ticker], label=f"{ticker} 30-day Volatility")
plt.title("30-Day Rolling Volatility")
plt.xlabel("Date")
plt.ylabel("Volatility")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
def adf_test(series, title=''):
    print(f"\nAugmented Dickey-Fuller Test: {title}")
    result = adfuller(series.dropna())
    labels = ['ADF Statistic', 'p-value', '# Lags Used', 'Number of Observations Used']
    out = pd.Series(result[0:4], index=labels)
    for key, val in result[4].items():
        out[f'Critical Value ({key})'] = val
    print(out)
    if result[1] < 0.05:
        print("✅ The series is stationary.")
    else:
        print("❌ The series is NOT stationary.")

for ticker in tickers:
    adf_test(adj_close[ticker], title=ticker)

In [ ]:
VaR_95 = daily_returns.quantile(0.05)
sharpe_ratios = (daily_returns.mean() / daily_returns.std()) * np.sqrt(252)

summary = pd.DataFrame({
    "VaR_95": VaR_95,
    "Sharpe Ratio": sharpe_ratios
})

print("\nRisk Metrics Summary:")
print(summary)